# Model Tutorial: XGBoost

The purpose of this notebook is to demonstrate how to train and predict XGBoost models used in this project. First, we will demonstrate the basic code, and then reproduce the results using a custom class `XGB` to make the code consistent for multiple models.

## Model Description

The goal is to forecast fuel moisture based on atmospheric data observations using machine learning models. The inputs include equilibrium moisture, calculated from relative humidity and surface temperature, collected from RAWS ground-based stations.

XGBoost is a variety of machine learning model that maps an input matrix of features to an output vector. This method can model regression problems, where the target output vector is a continuous quantity. Rows of the feature matrix are observed meteorological quantities at a certain location and time. Values of the output vector are observed fuel moisture quantities at corresponding locations and times.

XGBoost is a form of gradient boosting, which uses an ensemble of "weak learners" to model the target input. 

The final model outputs are time series of fuel moisture predictions. The model accuracy is calculated by comparing predicted fuel moisture to observed fuel moisture *at future times* and *at unobserved locations*.

## Setup

In [ ]:
import sys
sys.path.append('../src')
import pandas as pd
import numpy as np
import xgboost as xg
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import yaml
# Local modules
from fmda_models import XGB
import reproducibility

## Read and Split Data

In [ ]:
df = pd.read_pickle("../data/rocky_2023_05-09.pkl")
# Remove NA fm
df = df.dropna(subset=['fm'])

In [ ]:
# Set seed for reproducibility
reproducibility.set_seed(123)

# Create Data
X_train, X_test, y_train, y_test = train_test_split(df[["Ed", "Ew"]], df['fm'], test_size=.2)

## Manually Code XGBoost

In [ ]:
# create model instance
bst = xg.XGBRegressor(max_depth=3, eta=.1, min_child_weight=1, subsample=0.8, colsample_bytree=0.8,scale_pos_weight=1,
                      objective='reg:squarederror')
# fit model
bst.fit(X_train, y_train)

In [ ]:
fitted = bst.predict(X_train)
preds = bst.predict(X_test)

In [ ]:
# Calculate RMSE for the training data
rmse_train = np.sqrt(mean_squared_error(y_train, fitted))

# Calculate R-squared for the training data
r2_train = r2_score(y_train, fitted)

# Calculate RMSE for the test data
rmse_test = np.sqrt(mean_squared_error(y_test, preds))

# Calculate R-squared for the test data
r2_test = r2_score(y_test, preds)

print("RMSE for training data:", rmse_train)
print("R-squared for training data:", r2_train)
print("RMSE for test data:", rmse_test)
print("R-squared for test data:", r2_test)

## Reproduce using XGB Class

We now use a class `XGB` that reproduces the code above. The purpose of the class is to have different machine learning models with the same methods for concise code.

The `XGB` class accepts a dictionary for hyperparameters, which can be found in the file `params.yml`.

In [ ]:
with open('params.yaml', 'r') as file:
    all_params = yaml.safe_load(file)

params = all_params["xgb"]
params

In [ ]:
# Set seed for reproducibility
reproducibility.set_seed(123)

model = XGB(loss='reg:squarederror',params=params)
model.fit(X_train, y_train)
fitted = model.predict(X_train)
preds = model.predict(X_test)

In [ ]:
# Calculate RMSE for the training data
rmse_train = np.sqrt(mean_squared_error(y_train, fitted))

# Calculate R-squared for the training data
r2_train = r2_score(y_train, fitted)

# Calculate RMSE for the test data
rmse_test = np.sqrt(mean_squared_error(y_test, preds))

# Calculate R-squared for the test data
r2_test = r2_score(y_test, preds)

print("RMSE for training data:", rmse_train)
print("R-squared for training data:", r2_train)
print("RMSE for test data:", rmse_test)
print("R-squared for test data:", r2_test)

## Using Custom Loss

With XGBoost, we need a custom objective function to return the gradient and hessian: [XGBoost Documentation](https://xgboost.readthedocs.io/en/stable/tutorials/custom_metric_obj.html#customized-objective-function). However, `XGBoost` does support weights, so weighted versions of standard loss functions are easily coded.

Consider the weighted mean squared error loss function. We define:
* $N$: total number of observations
* $y^{(i)}_{pred}$: predicted value for $i$th observation from the model
* $y^{(i)}_{true}$: observed ("true") value for $i$th observation from the model
* $w_i$: weight for $i$th observation

$$
\frac{1}{N}\sum_{i=1}^N w_i(y^{(i)}_{pred}-y^{(i)}_{true})^2
$$



In [ ]:
weights = tf.exp(tf.multiply(-0.01, y_train))

In [ ]:
reproducibility.set_seed(123)
# create model instance
bst2 = xg.XGBRegressor(max_depth=3, eta=.1, min_child_weight=1, subsample=0.8, colsample_bytree=0.8,scale_pos_weight=1,
                      objective='reg:squarederror')
# fit model
bst2.fit(X_train, y_train, sample_weight = weights)

fitted = bst2.predict(X_train)
preds = bst2.predict(X_test)

In [ ]:
# Calculate RMSE for the training data
rmse_train = np.sqrt(mean_squared_error(y_train, fitted))

# Calculate R-squared for the training data
r2_train = r2_score(y_train, fitted)

# Calculate RMSE for the test data
rmse_test = np.sqrt(mean_squared_error(y_test, preds))

# Calculate R-squared for the test data
r2_test = r2_score(y_test, preds)

print("RMSE for training data:", rmse_train)
print("R-squared for training data:", r2_train)
print("RMSE for test data:", rmse_test)
print("R-squared for test data:", r2_test)

In [ ]:
reproducibility.set_seed(123)
model = XGB(loss='reg:squarederror',params=params)
model.fit(X_train, y_train, weights)
fitted = model.predict(X_train)
preds = model.predict(X_test)

In [ ]:
# Calculate RMSE for the training data
rmse_train = np.sqrt(mean_squared_error(y_train, fitted))

# Calculate R-squared for the training data
r2_train = r2_score(y_train, fitted)

# Calculate RMSE for the test data
rmse_test = np.sqrt(mean_squared_error(y_test, preds))

# Calculate R-squared for the test data
r2_test = r2_score(y_test, preds)

print("RMSE for training data:", rmse_train)
print("R-squared for training data:", r2_train)
print("RMSE for test data:", rmse_test)
print("R-squared for test data:", r2_test)